In [33]:
import glob
from collections import defaultdict
import pandas as pd
import os.path as op
import math
import os 
import sys
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

import sqlite3
import sqlalchemy

def safe_makedir(dname):
    """
    Make a directory if it doesn't exist, handling concurrent race conditions.
    """
    if not dname:
        return dname
    num_tries = 0
    max_tries = 5
    while not os.path.exists(dname):
        try:
            os.makedirs(dname)
        except OSError:
            if num_tries > max_tries:
                raise
            num_tries += 1
            time.sleep(2)
    return dname

# import my custom functions
sys.path.insert(0, '/home/jmunson-mcgee/')
from JMM_functions import *

pd.set_option("display.max_rows", 6)

outdir='/mnt/scgc/simon/microg2p/Melody/BLM1/Analyses/RNA_GORG_recruit/Summary_files/'
safe_make_dir(outdir)
os.chdir(outdir)

images='/mnt/scgc/simon/microg2p/Melody/BLM1/images/'
safe_make_dir(images)
    
matplotlib.__version__



Folder already exists
Folder already exists


'3.1.0'

In [34]:
libraries={'cDNA1-A01':'No_amendment_transcripts', 'cDNA2-A01':'Acetate_transcripts', 'cDNA3-A01':'CO_transcripts', 
         'cDNA4-A01':'H2_transcripts', 'cDNA7-A01':'Manifold_transcripts', 'cDNA6-A01':'cDNA6_control', 'cDNA9-A01':'cDNA9_control'}

in_files=glob.glob('/mnt/scgc/simon/microg2p/Melody/BLM1/Analyses/RNA_GORG_recruit/Summary_files/*_joined_annotated_RNA_GORG_results.csv')



In [35]:
DF_master=pd.DataFrame()

for file in in_files:
    base=os.path.basename(file)
    sample=base[:9]
    amend=libraries[sample]
    print(sample)
    
    DF_tmp=pd.read_csv(file, index_col=0)
    DF_tmp['amendment']=amend
    n = DF_tmp.gene_cluster_rep.nunique()
    print('the number of unique gene clusters is', n)
    
    print(len(DF_tmp))
    #if len(DF_tmp)>1
    DF_master=pd.concat([DF_master, DF_tmp], ignore_index=True)
    
DF_master.to_csv(os.path.join(outdir, 'BLM1_combined_GORG_recruitments.csv'))

DF_master

cDNA4-A01
the number of unique gene clusters is 11
11
cDNA2-A01
the number of unique gene clusters is 12
14
cDNA3-A01
the number of unique gene clusters is 2
2
cDNA7-A01
the number of unique gene clusters is 140
153
cDNA1-A01
the number of unique gene clusters is 14
16


/mnt/scgc_nfs/opt/common/anaconda3a/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()


,0,amendment,gene_cluster_rep,number_of_reads,prokka_EC_number,prokka_product,swissprot_CAZy,swissprot_EC_number,swissprot_KO,swissprot_Pfam,swissprot_TIGRFAMs,swissprot_eggNOG,swissprot_gene,taxonomic_lineage
0,NaN,H2_transcripts,AM-294-G05_N10;1964;2641,28722.0,NaN,hypothetical protein,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Archaea; Hadarchaeota; Hadarchaeia; Hadarchaea...
1,NaN,H2_transcripts,AM-294-I22_N59;2566;2784,66.0,NaN,hypothetical protein,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Archaea; Hadarchaeota; Hadarchaeia; Hadarchaea...
2,NaN,H2_transcripts,AM-294-G10_N3;30105;32429,18923.0,1.17.1.4,Probable xanthine dehydrogenase subunit D (XDH...,NaN,NaN,K00087,"PF01315,CPF02738",TIGR03196,"ENOG4105CE2,CCOG1529",NaN,Bacteria; Bipolaricaulota; Bipolaricaulia; Bip...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,NaN,No_amendment_transcripts,AM-294-P18_N23;17116;17334,1.0,NaN,Putative nitrogen fixation protein YutI,NaN,NaN,NaN,PF01106,NaN,"ENOG41083MR,CCOG0694",NaN,Bacteria; Acidobacteriota; Aminicenantia; Amin...
194,NaN,No_amendment_transcripts,AM-294-J06_N23;5746;6825,1.0,6.3.5.5,Carbamoyl-phosphate synthase pyrimidine-specif...,NaN,NaN,NaN,"PF00988,CPF00117",TIGR01368,NaN,NaN,Bacteria; Firmicutes_B; GCA-003054495; Carboxy...
195,NaN,No_amendment_transcripts,AM-296-B22_N14;2166;2447,1.0,NaN,hypothetical protein,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bacteria; Nitrospirota; Thermodesulfovibrionia...


In [43]:
n = DF_master.gene_cluster_rep.nunique()
print('the number of unique gene clusters is', n)

DF_master.groupby(['gene_cluster_rep', 'taxonomic_lineage']).ngroups

the number of unique gene clusters is 173


191

In [42]:
DF_Cas9=DF_master.copy()
DF_Cas9['prokka_product']=DF_Cas9['prokka_product'].fillna('hypothetical protein')
DF_Cas9=DF_Cas9[DF_Cas9['prokka_product'].str.contains('Cas9')]
DF_Cas9.to_csv(os.path.join(outdir, 'BLM1_cas9_transcripts'))
DF_Cas9

,0,amendment,gene_cluster_rep,number_of_reads,prokka_EC_number,prokka_product,swissprot_CAZy,swissprot_EC_number,swissprot_KO,swissprot_Pfam,swissprot_TIGRFAMs,swissprot_eggNOG,swissprot_gene,taxonomic_lineage
70,NaN,Manifold_transcripts,AM-294-N05_N9;10214;11533,22600.0,3.1.-.-,CRISPR-associated endonuclease Cas9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bacteria; Firmicutes_B; Desulfotomaculia; Ammo...
97,NaN,Manifold_transcripts,AM-294-N05_N9;10214;11533,76.0,3.1.-.-,CRISPR-associated endonuclease Cas9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bacteria; CSP1-3; NA; NA; SpSt-165; NA; SpSt-1...
149,NaN,Manifold_transcripts,AM-294-N20_N7;20661;21965,4.0,3.1.-.-,CRISPR-associated endonuclease Cas9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bacteria; Firmicutes_B; Desulfotomaculia; Ammo...
